In [1]:
import numpy as np

In [2]:
import pandas as pd

In [18]:
df = pd.read_csv('spam.csv', encoding='latin1' )

In [16]:
#data cleaning
#data analysis
#text preprocessing
# model building
# evaluation
#improvement
#website
#deployment

# Data Cleaning

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [23]:
#as the last three column has mostly data missing hence we will just drop those three columns

In [24]:
df.drop(columns = ['Unnamed: 2' , 'Unnamed: 3' , 'Unnamed: 4'] , inplace = True)

In [26]:
#now we will rename the col v1 and v2
df.rename(columns = {'v1' : 'target' , 'v2' : 'text'} , inplace = True)

In [27]:
df.sample(5)

,target,text
5014,ham,I think the other two still need to get cash b...
2704,spam,FreeMsg: Fancy a flirt? Reply DATE now & join ...
626,ham,How many licks does it take to get to the cent...
3411,ham,No she didnt. I will search online and let you...
1160,ham,You sure your neighbors didnt pick it up


In [28]:
# now what we want is to convert ham and spam in numerical value
# spam -> 1 and ham -> 0

In [33]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [110]:
df['target'] = encoder.fit_transform(df['target'])

C:\Users\FIRDAUS JAWED\AppData\Local\Temp\ipykernel_11776\1103932564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = encoder.fit_transform(df['target'])


In [111]:
# check the missing values

df.isnull().sum()

target              0
text                0
num_charecters      0
num_words           0
num_sentence        0
transformed_text    0
dtype: int64

In [112]:
# check the duplicate values
df.duplicated().sum()

0

In [113]:
# now remove them
df = df.drop_duplicates(keep = 'first')

# Data Analysis

In [114]:
# kitna spam h aur kitna ham h

In [115]:
df['target'].value_counts()

0    4516
1     653
Name: target, dtype: int64

In [116]:
# hence data is imbalance

In [117]:
# kitna sentences, charecter, word use hora 

In [118]:
!pip install nltk

In [119]:
import nltk

In [120]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\FIRDAUS
[nltk_data]     JAWED\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [121]:
# creating a new column in which there will be the amount of total charecters in the sms present
df['num_charecters'] = df['text'].apply(len)

# now creating a new column in which there will be the amount of total words in the sms present

df['num_words'] = df['text'].apply(lambda x : len(nltk.word_tokenize(x)))

# now creating a new column in which there will be the amount of total words in the sms present

df['num_sentence'] = df['text'].apply(lambda x : len(nltk.sent_tokenize(x)))

In [122]:
df.head()

,target,text,num_charecters,num_words,num_sentence,transformed_text
0,0,"Go until jurong point, crazy.. Available only ...",111,24,2,go jurong point crazi avail bugi n great world...
1,0,Ok lar... Joking wif u oni...,29,8,2,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,37,2,free entri 2 wkli comp win fa cup final tkt 21...
3,0,U dun say so early hor... U c already then say...,49,13,1,u dun say earli hor u c alreadi say
4,0,"Nah I don't think he goes to usf, he lives aro...",61,15,1,nah think goe usf live around though


# Data preprocessing

- lower case
- tokenization
- removing special charecters
- removing the stop words
- stemming

In [123]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to C:\Users\FIRDAUS
[nltk_data]     JAWED\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [124]:
import string

In [125]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [126]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
            
    text = y[:]
    y.clear()
    
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in text:
        y.append(ps.stem(i))
        
    return " ".join(y)    

In [127]:
df['transformed_text'] = df['text'].apply(transform_text)

# Building the model

In [128]:
# we are using naive bayes

In [129]:
# vectorize the text column in 

In [142]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features = 3000)

In [143]:
X = tfidf.fit_transform(df['transformed_text']).toarray()

In [144]:
y = df['target'].values

In [145]:
from sklearn.model_selection import train_test_split

In [146]:
X_train, X_test, y_train, y_test = train_test_split(X , y , test_size = 0.2 , random_state = 2)

In [147]:
from sklearn.naive_bayes import GaussianNB , MultinomialNB , BernoulliNB
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score

In [148]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [149]:
gnb.fit(X_train , y_train)
y_pred1 = gnb.predict(X_test)
print(accuracy_score(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))
print(precision_score(y_test,y_pred1))

0.8694390715667312
[[788 108]
 [ 27 111]]
0.5068493150684932


In [150]:
mnb.fit(X_train , y_train)
y_pred2 = mnb.predict(X_test)
print(accuracy_score(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))
print(precision_score(y_test,y_pred2))

0.9709864603481625
[[896   0]
 [ 30 108]]
1.0


In [151]:
bnb.fit(X_train , y_train)
y_pred3 = bnb.predict(X_test)
print(accuracy_score(y_test,y_pred3))
print(confusion_matrix(y_test,y_pred3))
print(precision_score(y_test,y_pred3))

0.9835589941972921
[[895   1]
 [ 16 122]]
0.991869918699187


In [152]:
# as the precision score of MBM is highest, hence we will use it as the data is imbalance hence accuracy score is secondary

In [153]:
import pickle
pickle.dump(tfidf , open('vectorizer.pkl' , 'wb'))
pickle.dump(mnb , open('model.pkl' , 'wb'))